In [1]:
# ArnoldTongues_vdP.jl
using DifferentialEquations
using LinearAlgebra
using Statistics
using Random
using CairoMakie           # high-quality plotting
using Base.Threads
using BenchmarkTools

In [2]:
# Van der Pol forced: x'' - μ(1 - x^2) x' + x = A*cos(ω t)
function vdp!(du,u,p,t)
    μ, A, ω = p
    x = @inbounds u[1]
    y = @inbounds u[2]
    @inbounds du[1] = y
    @inbounds du[2] = μ*(1 - x^2)*y - x + A * cos(ω * t)
    return nothing
end

vdp! (generic function with 1 method)

In [3]:
# compute stroboscopic samples: returns matrix samples (2 x Nsample)
function stroboscopic_samples(μ, A, ω; 
        u0 = [0.1, 0.0],
        n_transient = 200,
        n_sample = 200,
        reltol = 1e-8,
        abstol = 1e-10)

    T = 2π/ω
    tspan = (0.0, T * (n_transient + n_sample))

    p = (μ, A, ω)
    prob = ODEProblem(vdp!, u0, tspan, p)
    # save only at multiples of T after the transient
    save_times = T .* (n_transient+1 : n_transient + n_sample)
    sol = solve(prob, Tsit5(),
                saveat = save_times,
                reltol = reltol, abstol = abstol)

    # sol.u is vector of SVector or Vector{Float64}; convert to matrix 2 x n_sample
    X = reduce(hcat, sol.u)  # 2 x n_sample
    return X, T * n_sample
end

stroboscopic_samples (generic function with 1 method)

In [4]:
# estimate oscillator frequency from samples via unwrapped phase
function estimate_frequency_from_samples(X, total_time)
    # X: 2 x N matrix: rows [x; y]
    xs = X[1, :]
    ys = X[2, :]
    phases = atan.(ys, xs)                      # wrapped in (-π, π]
    # unwrap
    unwrapped = copy(phases)
    for i in 2:length(phases)
        d = unwrapped[i] - unwrapped[i-1]
        if d > π
            unwrapped[i:end] .-= 2π
        elseif d < -π
            unwrapped[i:end] .+= 2π
        end
    end
    # linear fit to phase vs time to get mean angular frequency
    Δphase = unwrapped[end] - unwrapped[1]
    ω_osc = Δphase / total_time   # rad/sec
    f_osc = ω_osc / (2π)
    return f_osc
end

estimate_frequency_from_samples (generic function with 1 method)

In [5]:
# find best rational p/q approximation with q <= qmax, tolerance tol (on ratio)
function detect_lock_ratio(ratio::Float64; qmax=8, tol=5e-3)
    # search denominators 1..qmax and numerators up to some range
    best = nothing
    for q in 1:qmax
        p = round(Int, ratio * q)
        if p <= 0
            continue
        end
        if abs(ratio - p/q) <= tol
            return (p, q)
        end
    end
    return nothing
end

detect_lock_ratio (generic function with 1 method)

In [6]:
# ---------------------------
# Parameter sweep (parallel)
# ---------------------------

function compute_arnold_tongues(; 
        μ = 1.0,                # vdp mu
        ωs = range(0.6, 1.6, length=241),     # drive frequency range (rad/s)
        As = range(0.0, 1.0, length=201),     # drive amplitude
        u0 = [0.1, 0.0],
        n_transient = 200,
        n_sample = 120,
        qmax = 8,
        tol = 5e-3)

    nω = length(ωs)
    nA = length(As)

    # result arrays
    pgrid = fill(0, nω, nA)
    qgrid = fill(0, nω, nA)
    ratiogrid = fill(NaN, nω, nA)

    @threads for i in 1:nω
        ω = ωs[i]
        for j in 1:nA
            A = As[j]
            try
                X, total_time = stroboscopic_samples(μ, A, ω; 
                                    u0 = u0, n_transient = n_transient, n_sample = n_sample)
                f_osc = estimate_frequency_from_samples(X, total_time)
                f_drive = ω / (2π)
                r = f_osc / f_drive
                # try to detect p:q
                pq = detect_lock_ratio(r; qmax = qmax, tol = tol)
                if pq !== nothing
                    pgrid[i, j] = pq[1]
                    qgrid[i, j] = pq[2]
                    ratiogrid[i, j] = pq[1] / pq[2]
                else
                    # optionally store nearest rational (or NaN)
                    ratiogrid[i, j] = r
                end
            catch e
                @warn "solver failed for i=$i j=$j ω=$ω A=$A: $e"
                pgrid[i, j] = 0
                qgrid[i, j] = 0
                ratiogrid[i, j] = NaN
            end
        end
    end

    return (ωs, As, pgrid, qgrid, ratiogrid)
end

compute_arnold_tongues (generic function with 1 method)

In [7]:
# ---------------------------
# Plotting helper
# ---------------------------

function plot_tongues(ωs, As, pgrid, qgrid; title="Arnold tongues (van der Pol)")
    # Map (p,q) → integer label
    labelgrid = zeros(Int, size(pgrid))
    labelmap = Dict{Tuple{Int,Int}, Int}()
    nextlabel = 1
    for i in axes(pgrid,1), j in axes(pgrid,2)
        p = pgrid[i,j]; q = qgrid[i,j]
        if p != 0 && q != 0
            key = (p,q)
            if !haskey(labelmap, key)
                labelmap[key] = nextlabel
                nextlabel += 1
            end
            labelgrid[i,j] = labelmap[key]
        else
            labelgrid[i,j] = 0
        end
    end

    fig = Figure(size = (900, 700))  # 'size' instead of 'resolution'
    ax = Axis(fig[1,1], xlabel = "drive frequency ω (rad/s)",
              ylabel = "drive amplitude A",
              title = title)

    hm = heatmap!(ax, ωs, As, labelgrid'; colormap = :viridis, interpolate = false)
    Colorbar(fig[1,2], hm, label = "p:q label")
    fig
end


plot_tongues (generic function with 1 method)

In [15]:
# ---------------------------
# Example run
# ---------------------------
ωs, As, pgrid, qgrid, ratiogrid = compute_arnold_tongues(
    μ = 1.0,
    ωs = range(0.6, 1.6, length=101), 
    As = range(0.0, 1.0, length=101),
    n_transient = 100,
    n_sample = 50,
    qmax = 7,
    tol = 4e-3
)

(0.6:0.01:1.6, 0.0:0.01:1.0, [0 0 … 0 0; 0 0 … 0 0; … ; 2 2 … 0 0; 2 2 … 0 0], [0 0 … 0 0; 0 0 … 0 0; … ; 5 5 … 0 0; 5 5 … 0 0], [0.4198415937185148 0.41987667333354994 … -1.1643964747492538e-12 -1.418846680283251e-12; 0.44336803825435456 0.44322521389500796 … -1.7196299695344328e-12 -1.0335716025240262e-12; … ; 0.4 0.4 … 0.39115062687582675 0.3911178514059466; 0.4 0.4 … 0.3951488997203655 0.3948183042293008])

In [16]:
fig = plot_tongues(ωs, As, pgrid, qgrid; title="Arnold tongues — forced van der Pol (μ=1)")
save("arnold_tongues_vdp.svg", fig)
println("Saved arnold_tongues_vdp.svg")

Saved arnold_tongues_vdp.svg
